# Amazon Data Collection

###### This project is meant to serve as an exercise for web scrapping and database creation. We begin with the retrieval of jean reviews from Amazon listings to create a database for future analysis.

###### After retrieving the data, we will perform data tranformation to ensure each column's use in Tableau data visualization. 

## Import Libraries

In [46]:
# standard library imports
import csv
import re
from datetime import datetime

# third-party imports
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd

## Initial Fucntion Definition

###### It may be best to refer to the ```main``` function before the following sections for better clarity. 
###### Obtaining the data we need involves 4 steps:
1. Retrieving the html source code
2. Dissecting the code for the desired attributes
3. Recording the data into a database
4. Exporting that database as a .csv file
 
###### These tasks have been implemented in the functions defined below. The ```get_html``` function below takes a search result item and returns the html for its product page.

In [3]:
def get_html(item):
    '''Return the HTML Source Code For An Item'''
    # First get the item
    atag = item.h2.a
    url = 'https://www.amazon.com' + atag.get('href')
    
    # Get page source for item
    driver = webdriver.Chrome(r'C:\Users\marti\Downloads\chromedriver.exe')
    driver.get(url)
    item_page = BeautifulSoup(driver.page_source, 'html.parser')
    
    return item_page

## Dataset Parameter Retrieval

###### ```get_jean_info``` uses the source code returned from ```get_html``` to retrieve product and review data. Each review is stored in a dictionary object that is then stored in the ```review_collection``` list. There are 8 data points or columns collected by this function:
 - Product Name
 - Product Price
 - Product Brand
 - Review Rating
 - Review Title
 - Review Body
 - Review Date
 - Product Size

###### With this list of data points, we need only a way to find and collect them. Within the function below, we define the code to navigate through each product page and nested review pages. Before we continue, take a moment to understand how our data is being extracted.

###### As an example, the HTML for the page below is used by the ```get_jean_info``` function for data collection. Product name and price are taken right from this page before using the "see all reviews" link near the page's end to access the "Customer Reviews" page.

![item_page](item_page.png)

![see_all](see_all.png)

###### From this link, the HTML from the "Customer Reviews" page is used to assign values for the remaining data points. If a value cannot be found, ```None``` is assigned instead. To ensure a decent amount of reviews are collected for each item, a maximum of 900 pages are searched for customer reviews.

In [ ]:


def get_jean_info(item_page):
    '''Obtain all product info for item'''
    
    # Get product name and price
    product_name = item_page.find('span', {'id': 'productTitle'}).text.strip() if item_page.find('span', {'id': 'productTitle'}) else None
    price = item_page.find('span', {'id': 'priceblock_ourprice'}).text.strip() if item_page.find('span', {'id': 'priceblock_ourprice'}) else None
    
    review_collection = []
    
    for x in range(1, 900):
        
        if not item_page.find('a', {'data-hook': 'see-all-reviews-link-foot'}):
            break
        else:
            pass

        
        # Get page source for review page
        all_reviews = 'https://www.amazon.com' + item_page.find('a', {'data-hook': 'see-all-reviews-link-foot'}).get('href') + f'&pageNumber={x}'
        driver.get(all_reviews)
        review_page = BeautifulSoup(driver.page_source, 'html.parser')
    
        # Get review data
        reviews = review_page.find_all('div', {'data-hook': 'review'})

        for i in reviews:
            review = {
                'product_name': product_name,
                'price': price,
                'review_rating': float(i.find('i', {'data-hook': 'review-star-rating'}).text.replace('out of 5 stars', '').strip()) if i.find('i', {'data-hook': 'review-star-rating'}) else None,
                'review_title': i.find('a', {'data-hook': 'review-title'}).text.strip() if i.find('a', {'data-hook': 'review-title'}) else None,
                'review_body': i.find('span', {'data-hook': 'review-body'}).text.strip() if i.find('span', {'data-hook': 'review-body'}) else None,
                'review_date': i.find('span', {'data-hook': 'review-date'}).text if i.find('span', {'data-hook': 'review-date'}) else None,
                
                'size': i.find('a', {'data-hook': 'format-strip'})
                .find_all(text=True, recursive=False)[0]
                .replace('Size: ', '').strip() if i
                .find('a', {'data-hook': 'format-strip'}) else None
            }
            review_collection.append(review)
        
        # If there isn't a 'Next page' button, break the loop
        if not review_page.find('li', {'class': 'a-last'}):
            break
        else:
            pass
    
        if not review_page.find('li', {'class': 'a-disabled a-last'}):
            pass
        else:
            break
        
    return review_collection

    

## Main Program Definition

###### Our ```main``` function uses the previous functions to create a csv file of review data. A maximum of eight pages of search results are used here. This process of opening the webdriver, retrieving html, and building the records is very time-consuming (hours worth). This is one of the biggest downsides to using a webdriver over an API.

In [4]:
def main(url):
    """Run main program routine"""
    
    # startup web driver
    driver = webdriver.Chrome(r'C:\Users\marti\Downloads\chromedriver.exe')
    
    records = []
    url += '&page={}'
    
    # A maximum of 8 pages are searched for product listings
    for page in range(1, 8):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in results:
            item_page = get_html(item)
            records.extend(get_jean_info(item_page))
            
            
    df = pd.DataFrame(records)
    df.to_csv('jean_reviews.csv', index=False)
    print('Done.')


In [ ]:
main('https://www.amazon.com/s?k=mens+jeans&rh=p_72%3A2661618011&s=review-rank&dc&qid=1629410817&rnid=2661617011&ref=sr_pg_1')

Done.


## Data Cleaning and Additional Parameter Definition

###### Now that we have successfully created an initial dataset, we should preview our results.

In [59]:
df = pd.read_csv('jean_reviews.csv')

df.head()

,product_name,price,brand,review_rating,review_title,review_body,review_date,size
0,Perry Ellis Men's Skinny Stretch Denim Jeans,$31.29 - $36.17,Perry Ellis,5.0,Worth the money,Excellent product totally recommended,"Reviewed in the United States on September 13,...",NaN
1,Perry Ellis Men's Skinny Stretch Denim Jeans,$31.29 - $36.17,Perry Ellis,5.0,Comfortable,Great jeans,"Reviewed in the United States on May 12, 2021",36W x 30L
2,Perry Ellis Men's Skinny Stretch Denim Jeans,$31.29 - $36.17,Perry Ellis,5.0,Fantastique!,Les jeans stretch de cette marque sont fantast...,"Reviewed in Canada on March 19, 2020",31W x 32L
3,Perry Ellis Men's Skinny Stretch Denim Jeans,$31.29 - $36.17,Perry Ellis,5.0,Super cómodos,Super cómodos,"Reviewed in Mexico on April 29, 2021",32W x 34L
4,Perry Ellis Men's Skinny Stretch Denim Jeans,$31.29 - $36.17,Perry Ellis,5.0,cubre mis expectativas,Como esperaba,"Reviewed in Mexico on April 2, 2021",34W x 30L


In [60]:
df_counts = df.product_name.value_counts()
p = []
for i,r in df_counts.items():
    (p.append(i))
    
count_row = df.shape[0]
count_col = df.shape[1]

In [61]:
print("#Unique Brands: ", len(p))
print("Row Count: ", count_row)
print("Column Count: ", count_col)

#Unique Brands:  243
Row Count:  277393
Column Count:  8


## Data Cleaning

###### To improve the usability of our dataset, we must create some additional columns and delete duplicate rows.

In [62]:
# Delete duplicate rows
df.drop_duplicates(subset="review_body", keep=False, inplace=True)

In [63]:
df_counts = df.product_name.value_counts()
p = []
for i,r in df_counts.items():
    (p.append(i))
    
count_row = df.shape[0]
count_col = df.shape[1]

In [64]:
print("#Unique Brands: ", len(p))
print("Row Count: ", count_row)
print("Column Count: ", count_col)

#Unique Brands:  231
Row Count:  118865
Column Count:  8


### Price

In [65]:
def get_price(price):
    '''Return jean price as a floating-point type using highest value in range'''
    x = re.findall(r"(?<=\ - \$)(\d+\.\d+)", str(price))
    y = x if x else re.findall(r"(?<=\$)(.*?\d+\.\d+)", str(price))
    
    match = float(y[0]) if y else None
    
    return match

In [66]:
df['prices'] = df['price'].apply(get_price)

In [67]:
def get_date(date):
    '''Return date as datetime object'''
    match = re.findall(r'(?<=on )(.*)', date)
    
    new_date = datetime.strptime(match[0], '%B %d, %Y')
    return new_date

In [68]:
df['review_dates'] = df['review_date'].apply(get_date)

In [69]:
def price_group(price):
    '''Return jean price group'''
    if (price >= 80):
        a = '$80+'
    elif (70 <= price < 80):
        a = '$70-$80'
    elif (60 <= price < 70):
        a = '$60-$70'
    elif (50 <= price < 60):
        a = '$50-$60'
    elif (40 <= price < 50):
        a = '$40-$50'
    elif (30 <= price < 40):
        a = '$30-$40'
    else:
        a = '>$30'
    return a

In [70]:
df['price_groups'] = df['prices'].map(price_group)

## Complete Dataset Review

In [71]:
df_counts = df.product_name.value_counts()
p = []
for i,r in df_counts.items():
    (p.append(i))
    
count_row = df.shape[0]
count_col = df.shape[1]

In [72]:
print("#Unique Brands: ", len(p))
print("Row Count: ", count_row)
print("Column Count: ", count_col)

#Unique Brands:  231
Row Count:  118865
Column Count:  11


## Save New Dataset

In [73]:
df.to_csv('jean_reviews2.csv', index=False)

## Tableau Dashboard
###### Here is our completed Tableau dashboard using our final dataset.

![Tableau Dashboard](dashboard_1.png)